In [1]:
import pandas as pd
import numpy as np

In [46]:
data_1=pd.read_csv('data_1.csv')

In [47]:
data_1.head()

,Unnamed: 0,online_order,book_table,votes,type,city,new_cuisines,rate_imputed,cost_imputed
0,0,Yes,Yes,775,Buffet,Banashankari,North Indian,good,800.0
1,1,Yes,Yes,775,Buffet,Banashankari,Mughlai,good,800.0
2,2,Yes,Yes,775,Buffet,Banashankari,Chinese,good,800.0
3,3,Yes,No,787,Buffet,Banashankari,Chinese,good,800.0
4,4,Yes,No,787,Buffet,Banashankari,North Indian,good,800.0


In [48]:
data_1.drop('Unnamed: 0', axis=1, inplace=True)

In [49]:
data_1.shape

(126819, 8)

In [50]:
data_1.head()

,online_order,book_table,votes,type,city,new_cuisines,rate_imputed,cost_imputed
0,Yes,Yes,775,Buffet,Banashankari,North Indian,good,800.0
1,Yes,Yes,775,Buffet,Banashankari,Mughlai,good,800.0
2,Yes,Yes,775,Buffet,Banashankari,Chinese,good,800.0
3,Yes,No,787,Buffet,Banashankari,Chinese,good,800.0
4,Yes,No,787,Buffet,Banashankari,North Indian,good,800.0


In [51]:
#data_1=data_1.sample(65000, random_state=34)

In [52]:
data_1.shape

(126819, 8)

In [53]:
data_1.head()

,online_order,book_table,votes,type,city,new_cuisines,rate_imputed,cost_imputed
0,Yes,Yes,775,Buffet,Banashankari,North Indian,good,800.0
1,Yes,Yes,775,Buffet,Banashankari,Mughlai,good,800.0
2,Yes,Yes,775,Buffet,Banashankari,Chinese,good,800.0
3,Yes,No,787,Buffet,Banashankari,Chinese,good,800.0
4,Yes,No,787,Buffet,Banashankari,North Indian,good,800.0


In [54]:
data_1.reset_index(drop=True, inplace=True)

In [55]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126819 entries, 0 to 126818
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   online_order  126819 non-null  object 
 1   book_table    126819 non-null  object 
 2   votes         126819 non-null  int64  
 3   type          112903 non-null  object 
 4   city          112903 non-null  object 
 5   new_cuisines  126819 non-null  object 
 6   rate_imputed  126819 non-null  object 
 7   cost_imputed  126819 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 7.7+ MB


In [56]:
data_1['type']=data_1['type'].fillna(data_1['type'].mode()[0])
data_1['city']=data_1['city'].fillna(data_1['city'].mode()[0])

In [57]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126819 entries, 0 to 126818
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   online_order  126819 non-null  object 
 1   book_table    126819 non-null  object 
 2   votes         126819 non-null  int64  
 3   type          126819 non-null  object 
 4   city          126819 non-null  object 
 5   new_cuisines  126819 non-null  object 
 6   rate_imputed  126819 non-null  object 
 7   cost_imputed  126819 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 7.7+ MB


In [58]:
from sklearn.compose import ColumnTransformer

In [59]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_1.drop(columns=['cost_imputed']), data_1['cost_imputed'] 
                                                    ,test_size=0.25,random_state=76)

In [60]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [61]:
transformer = ColumnTransformer(transformers = [
    ('oe',OrdinalEncoder(categories=[['poor','good',"verygood"]]),['rate_imputed']),
    ('ohe',OneHotEncoder(sparse=False,drop='first'),["online_order","book_table",'type','city','new_cuisines'])
], remainder = 'passthrough')

In [75]:
X_train = transformer.fit_transform(x_train)

In [77]:
X_test =transformer.transform(x_test)

### Model Training

#### Decision Tree

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [80]:
from sklearn.tree import DecisionTreeRegressor

In [81]:
dt = DecisionTreeRegressor()

In [85]:
%%time
dt.fit(X_train, y_train)

CPU times: total: 11.2 s
Wall time: 11.2 s


DecisionTreeRegressor()

In [83]:
dt.score(X_train, y_train)

0.9757954727817365

In [84]:
dt.score(X_test, y_test)

0.7543408630634844

##### DT Grid

In [154]:
from sklearn.model_selection import GridSearchCV

In [155]:
tree_para={
    'min_samples_leaf':[3,4,5,8],
    'max_depth':[20, 25, 30],
    'min_samples_split':[5,8,10]
    }

In [156]:
rf1=DecisionTreeRegressor()

In [157]:
gcv= GridSearchCV(rf1,tree_para,n_jobs=-1, verbose=2,cv=3)

In [160]:
gcv.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'max_depth': [20, 25, 30],
                         'min_samples_leaf': [3, 4, 5, 8],
                         'min_samples_split': [5, 8, 10]},
             verbose=2)

In [162]:
gcv.best_params_

{'max_depth': 30, 'min_samples_leaf': 3, 'min_samples_split': 5}

In [161]:
gcv.best_score_

0.7314346086017709

#### Random Forest

In [87]:
from sklearn.ensemble import RandomForestRegressor

In [137]:
%%time
rf=RandomForestRegressor(n_estimators=20,max_depth=38,min_samples_split=5, n_jobs=-1, verbose=2)

CPU times: total: 0 ns
Wall time: 0 ns


In [138]:
%%time
rf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 20building tree 2 of 20
building tree 3 of 20building tree 4 of 20


building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20
CPU times: total: 3min 40s
Wall time: 1min 6s


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.1min finished


RandomForestRegressor(max_depth=38, min_samples_split=5, n_estimators=20,
                      n_jobs=-1, verbose=2)

In [139]:
rf.score(X_train, y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.2s finished


0.8952383808072021

In [140]:
rf.score(X_test, y_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished


0.8132939216688841

In [141]:
rf_y_pred=rf.predict(X_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.0s finished


In [143]:
print(mean_absolute_error(y_test,rf_y_pred))

120.55898250936795


##### RF with grid

In [110]:
from sklearn.model_selection import GridSearchCV

In [144]:
grid_param = {
    "max_depth":[36,37,38],
    "n_estimators" : [15,20,25],
    'min_samples_leaf' : [4,5],
}

In [145]:
rf1=RandomForestRegressor()

In [146]:
gscv= GridSearchCV(estimator=rf1,param_grid=grid_param,cv=3,n_jobs=-1,verbose=2)

In [147]:
gscv.fit(X_train, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [36, 37, 38], 'min_samples_leaf': [4, 5],
                         'n_estimators': [15, 20, 25]},
             verbose=2)

In [148]:
gscv.best_params_

{'max_depth': 37, 'min_samples_leaf': 4, 'n_estimators': 25}

In [150]:
rf_hp=RandomForestRegressor(max_depth= 37, min_samples_leaf=4, n_estimators=25)

In [151]:
rf_hp.fit(X_train,y_train)

RandomForestRegressor(max_depth=37, min_samples_leaf=4, n_estimators=25)

In [152]:
rf_hp.score(X_train,y_train)

0.8500850118879782

In [153]:
rf_hp.score(X_test,y_test)

0.793944048748963

In [163]:
gscv.best_score_

0.767045153750325